# Query -> Category

<small>
(from <a href="http://maven.com/softwaredoug/cheat-at-search">Cheat at Search with LLMs</a> training course by Doug Turnbull.)
</small>

**Refinement** -- Use the -full list- of classifications, and allow a list to be returned

We learned that we want to try to model aspects of the user's _information need_ not just make queries better.

One such common dimension is query -> category classification

## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive)

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-k0s64jok
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-k0s64jok
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 7fbf2bf2845343912918f337a29127f9edc50bd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.3 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=28125 sha256=d79ddefa4e5881651692ac6a334752165c4fd3a5b606f8abee953f566b5d58ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-275btbf_/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981d9

INFO:data_dir:Mounting data directory at /content/drive/MyDrive/cheat-at-search-data/


## Import helpers

Helpers to compute ndcg of each query and other comparison functions

In [ ]:
from cheat_at_search.data_dir import ensure_data_subdir
from cheat_at_search.search import run_strategy, graded_bm25, ndcgs, ndcg_delta, vs_ideal

2025-08-31 21:07:05,507 - data_dir - INFO - Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


INFO:data_dir:Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


2025-08-31 21:07:27,697 - data_dir - INFO - Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


INFO:data_dir:Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


2025-08-31 21:07:30,270 - data_dir - INFO - Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


...

INFO:data_dir:Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


2025-08-31 21:07:34,838 - data_dir - INFO - Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


INFO:data_dir:Directory /content/drive/MyDrive/cheat-at-search-data/wands_enriched already exists. Checking for updates...


2025-08-31 21:07:36,235 - data_dir - INFO - Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


INFO:data_dir:Updated https://github.com/softwaredoug/WANDS.git dataset at /content/drive/MyDrive/cheat-at-search-data/wands_enriched


## Import WANDS data

Import [Wayfair Annotated Dataset](https://github.com/wayfair/WANDS) a labeled furniture e-commerce dataset

In [ ]:
from cheat_at_search.wands_data import products

products

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,category,sub_category,cat_subcat
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",Browse By Brand,All-Clad,Browse By Brand / All-Clad
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"[basematerialdetails : steel, : gray wood, of...",Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,[additionaltoolsrequirednotincluded : power dr...,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0,"[legmaterialdetails : rubberwood, backheight-s...",Furniture,Living Room Furniture,Furniture / Living Room Furniture


## Query -> Full classification

We'll first setup the model of query -> category and subcategory. We've precurated categories / subcategories for you to help.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
from cheat_at_search.enrich import AutoEnricher

from typing import Literal, get_args
FullyQualifiedClassifications = Literal[
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs',
 'Rugs / Area Rugs',
 'Furniture / Office Furniture / Desks',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / End & Side Tables',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows',
 'Furniture / Bedroom Furniture / Dressers & Chests',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Conversation Sets',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities',
 'Furniture / Living Room Furniture / Console Tables',
 'Décor & Pillows / Art / All Wall Art',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs',
 'Furniture / Office Furniture / Office Chairs',
 'Décor & Pillows / Mirrors / All Mirrors',
 'Bed & Bath / Bedding / All Bedding',
 'Décor & Pillows / Wall Décor / Wall Accents',
 'Furniture / Living Room Furniture / Chairs & Seating / Recliners',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen and Dining Sets',
 'Décor & Pillows / Window Treatments / Curtains & Drapes',
 'Furniture / Living Room Furniture / Sectionals',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Kids Beds',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / TV Stands & Entertainment Centers',
 'Lighting / Ceiling Lights / Chandeliers',
 'Furniture / Bedroom Furniture / Nightstands',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Kids Desks',
 'Décor & Pillows / Home Accessories / Decorative Objects',
 'Furniture / Bedroom Furniture / Beds & Headboards / Headboards',
 'Furniture / Living Room Furniture / Sofas',
 'Furniture / Living Room Furniture / Cabinets & Chests',
 'Décor & Pillows / Clocks / Wall Clocks',
 'Storage & Organization / Bathroom Storage & Organization / Bathroom Cabinets & Shelving',
 'Lighting / Table & Floor Lamps / Table Lamps',
 'Furniture / Living Room Furniture / Ottomans & Poufs',
 'Furniture / Kitchen & Dining Furniture / Kitchen Islands & Carts',
 'Furniture / Living Room Furniture / Bookcases',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Sofas & Sectionals',
 'Furniture / Office Furniture / Office Storage Cabinets',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Tables',
 'Contractor / Entry & Hallway / Coat Racks & Umbrella Stands',
 'Bed & Bath / Bedding Essentials / Mattress Pads & Toppers',
 'Home Improvement / Hardware / Home Hardware / Switch Plates',
 'Baby & Kids / Toddler & Kids Playroom / Playroom Furniture / Toddler & Kids Chairs & Seating',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Outdoor Covers / Patio Furniture Covers',
 'Rugs / Doormats',
 'Rugs / Kitchen Mats',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Queen Size Beds',
 'Furniture / Bedroom Furniture / Daybeds',
 'Furniture / Living Room Furniture / Living Room Sets',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Dining Sets',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Single Hole Bathroom Sink Faucets',
 'Outdoor / Outdoor Décor / Statues & Sculptures',
 'Décor & Pillows / Art / All Wall Art / Green Wall Art',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Table Sets',
 'Furniture / Living Room Furniture / Chairs & Seating / Chaise Lounge Chairs',
 'Storage & Organization / Wall Shelving & Organization / Wall and Display Shelves',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Rectangle Coffee Tables',
 'Décor & Pillows / Art / All Wall Art / Brown Wall Art',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools / Counter (24-27) Bar Stools & Counter Stools',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Plant Stands & Tables',
 'Décor & Pillows / Window Treatments / Curtain Hardware & Accessories',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs / Side Kitchen & Dining Chairs',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Outdoor Club Chairs',
 'Furniture / Living Room Furniture / Chairs & Seating / Benches',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Sinks / Farmhouse & Apron Kitchen Sinks',
 'Kitchen & Tabletop / Kitchen Organization / Food Pantries',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel & Robe Hooks / Black Towel & Robe Hooks',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Deck Boxes & Patio Storage',
 'Outdoor / Garden / Planters',
 'Lighting / Wall Lights / Bathroom Vanity Lighting',
 'Furniture / Kitchen & Dining Furniture / Sideboards & Buffets',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Storage Racks & Shelving Units',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Bronze Cabinet & Drawer Pulls',
 'Storage & Organization / Storage Containers & Drawers / All Storage Containers',
 'Bed & Bath / Shower Curtains & Accessories / Shower Curtains & Shower Liners',
 'Storage & Organization / Bathroom Storage & Organization / Hampers & Laundry Baskets',
 'Lighting / Light Bulbs & Hardware / Light Bulbs / All Light Bulbs / LED Light Bulbs',
 'Décor & Pillows / Art / All Wall Art / Blue Wall Art',
 'Bed & Bath / Mattresses & Foundations / Innerspring Mattresses',
 'Lighting / Outdoor Lighting / Outdoor Wall Lighting',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Natural Material Storage / Log Storage',
 'Bed & Bath / Bathroom Accessories & Organization / Countertop Bath Accessories',
 'Storage & Organization / Shoe Storage / All Shoe Storage',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Ceramic Floor Tiles & Wall Tiles',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Black Cabinet & Drawer Pulls',
 'Bed & Bath / Mattresses & Foundations / Adjustable Beds',
 "Rugs / Area Rugs / 2' x 3' Area Rugs",
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Twin Beds',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Widespread Bathroom Sink Faucets',
 "Rugs / Area Rugs / 4' x 6' Area Rugs",
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets',
 'Kitchen & Tabletop / Tableware & Drinkware / Table & Kitchen Linens / All Table Linens',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Food Storage Containers',
 'Décor & Pillows / Flowers & Plants / Faux Flowers',
 'Bed & Bath / Bedding / All Bedding / Twin Bedding',
 'Furniture / Bedroom Furniture / Dressers & Chests / White Dressers & Chests',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Porcelain Floor Tiles & Wall Tiles',
 'Home Improvement / Flooring, Walls & Ceiling / Flooring Installation & Accessories / Molding & Millwork / Wall Molding & Millwork',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Barn Door Hardware',
 'Bed & Bath / Bedding / Sheets & Pillowcases',
 'Furniture / Office Furniture / Chair Mats / Hard Floor Chair Mats',
 'Outdoor / Outdoor Fencing & Flooring / All Fencing',
 'Storage & Organization / Closet Storage & Organization / Clothes Racks & Garment Racks',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Colanders, Strainers, & Salad Spinners',
 'Outdoor / Hot Tubs & Saunas / Saunas',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Blue Throw Pillows',
 'Bed & Bath / Bedding Essentials / Bed Pillows',
 'Lighting / Wall Lights / Wall Sconces',
 'Outdoor / Front Door Décor & Curb Appeal / Mailboxes',
 'Outdoor / Garden / Greenhouses',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Faucets & Systems',
 'Bed & Bath / Mattresses & Foundations / Queen Mattresses',
 'Furniture / Bedroom Furniture / Jewelry Armoires',
 'Outdoor / Outdoor Shades / Awnings',
 'Baby & Kids / Nursery Bedding / Crib Bedding Sets',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Knobs / Brass Cabinet & Drawer Knobs',
 'Décor & Pillows / Art / All Wall Art / Red Wall Art',
 'Lighting / Ceiling Lights / All Ceiling Lights',
 'Lighting / Light Bulbs & Hardware / Lighting Components',
 'Furniture / Game Tables & Game Room Furniture / Poker & Card Tables',
 'Appliances / Kitchen Appliances / Range Hoods / All Range Hoods',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Natural Stone Floor Tiles & Wall Tiles',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools / Bar (28-33) Bar Stools & Counter Stools',
 'Outdoor / Outdoor Cooking & Tableware / Outdoor Serving & Tableware / Coolers, Baskets & Tubs / Picnic Baskets & Backpacks',
 'Décor & Pillows / Picture Frames & Albums / All Picture Frames',
 'Bed & Bath / Shower Curtains & Accessories / Shower Curtain Hooks',
 'Outdoor / Outdoor Shades / Outdoor Umbrellas / Patio Umbrella Stands & Bases',
 'Outdoor / Outdoor & Patio Furniture / Patio Bar Furniture / Patio Bar Stools',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Toilets & Bidets / Toilet Paper Holders / Free Standing Toilet Paper Holders',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Bike & Sport Racks',
 'Appliances / Kitchen Appliances / Refrigerators & Freezers / All Refrigerators / French Door Refrigerators',
 'Décor & Pillows / Home Accessories / Decorative Trays',
 'School Furniture and Supplies / School Spaces / Computer Lab Furniture / Podiums & Lecterns',
 'Lighting / Light Bulbs & Hardware / Lighting Shades',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Home Bars & Bar Sets',
 'Lighting / Table & Floor Lamps / Floor Lamps',
 'Décor & Pillows / Wall Décor / Wall Accents / Brown Wall Accents',
 'Kitchen & Tabletop / Small Kitchen Appliances / Pressure & Slow Cookers / Slow Cookers / Slow Slow Cookers',
 'Décor & Pillows / Window Treatments / Curtains & Drapes / 90 Inch Curtains & Drapes',
 'Furniture / Bedroom Furniture / Armoires & Wardrobes',
 'Kitchen & Tabletop / Tableware & Drinkware / Flatware & Cutlery / Serving Utensils',
 'Baby & Kids / Baby & Kids Décor & Lighting / All Baby & Kids Wall Art',
 'Furniture / Office Furniture / Desks / Writing Desks',
 'Furniture / Office Furniture / Office Chairs / Task Office Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Shower & Bathtub Doors',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Rocking Chairs & Gliders',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Wall Paneling',
 'Outdoor / Garden / Plant Stands & Accessories',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Tables / 4 Seat Kitchen & Dining Tables',
 'Décor & Pillows / Home Accessories / Vases, Urns, Jars & Bottles',
 'Lighting / Wall Lights / Under Cabinet Lighting / Strip Under Cabinet Lighting',
 'Furniture / Bedroom Furniture / Bedroom and Makeup Vanities',
 'Pet / Dog / Dog Bowls & Feeding Supplies / Pet Bowls & Feeders',
 'Décor & Pillows / Candles & Holders / Candle Holders',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Shower & Bathtub Accessories',
 'Furniture / Office Furniture / Office Chair Accessories / Seat Cushion Office Chair Accessories',
 'Furniture / Office Furniture / Chair Mats',
 'Furniture / Living Room Furniture / Chairs & Seating / Massage Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities / Modern & Contemporary Bathroom Vanities',
 'Lighting / Ceiling Fans / All Ceiling Fans',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Black Kitchen Faucets',
 'Lighting / Light Bulbs & Hardware / Light Bulbs / All Light Bulbs / Incandescent Light Bulbs',
 'Home Improvement / Flooring, Walls & Ceiling / Flooring Installation & Accessories / Molding & Millwork',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Bathtubs',
 'Décor & Pillows / Art / All Wall Art / Yellow Wall Art',
 'Pet / Dog / Pet Gates, Fences & Doors / Pet Gates',
 'Furniture / Bedroom Furniture / Beds & Headboards / Bed Frames / Twin Bed Frames',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel Bars, Racks, and Stands / Metal Towel Bars, Racks, and Stands',
 'Décor & Pillows / Art / All Wall Art / Pink Wall Art',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Smoke Detectors / Wall & Ceiling Mounted Smoke Detectors',
 'Outdoor / Garden / Planters / Plastic Planters',
 'Décor & Pillows / Mirrors / All Mirrors / Accent Mirrors',
 'Appliances / Kitchen Appliances / Range Hoods / All Range Hoods / Wall Mount Range Hoods',
 'Outdoor / Garden / Garden Décor / Lawn & Garden Accents',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Round Coffee Tables',
 'Kitchen & Tabletop / Tableware & Drinkware / Dinnerware / Dining Bowls',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads / Dual Shower Heads',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Glass Floor Tiles & Wall Tiles',
 'School Furniture and Supplies / Facilities & Maintenance / Trash & Recycling',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Nickel Cabinet & Drawer Pulls',
 'Storage & Organization / Closet Storage & Organization / Closet Systems',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Full & Double Beds',
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands / Printer Carts & Stands',
 'Storage & Organization / Closet Storage & Organization / Closet Accessories',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities / Traditional Bathroom Vanities',
 'Home Improvement / Plumbing / Core Plumbing / Parts & Components',
 'Holiday Décor / Christmas / Christmas Trees / All Christmas Trees',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Black Throw Pillows',
 'Furniture / Game Tables & Game Room Furniture / Sports Team Fan Shop & Memorabillia / Life Size Cutouts',
 'Lighting / Ceiling Lights / Pendant Lighting',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel & Robe Hooks',
 'Appliances / Washers & Dryers / Dryers / All Dryers / Gas Dryers',
 'Outdoor / Outdoor Recreation / Backyard Play / Kids Cars & Ride-On Toys',
 'Kitchen & Tabletop / Small Kitchen Appliances / Coffee, Espresso, & Tea / Coffee Makers',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Sofas & Sectionals / Sectional Patio Sofas & Sectionals',
 'Lighting / Wall Lights / Under Cabinet Lighting',
 'Foodservice / Foodservice Tables / Table Parts',
 'Lighting / Outdoor Lighting / Landscape Lighting / All Landscape Lighting / Fence Post Cap Landscape Lighting',
 'Lighting / Outdoor Lighting / Landscape Lighting / All Landscape Lighting',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Tables / All Patio Tables',
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands / Utility Carts & Stands',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Outdoor Chaise & Lounge Chairs',
 'Furniture / Living Room Furniture / Chairs & Seating / Recliners / Brown Recliners',
 'Pet / Bird / Bird Perches & Play Gyms',
 'Décor & Pillows / Picture Frames & Albums / All Picture Frames / Single Picture Picture Frames',
 'Lighting / Outdoor Lighting / Outdoor Lanterns & Lamps',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls',
 'Bed Accessories',
 'Clips/Clamps',
 'Décor & Pillows / Wall Décor / Wall Decals',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles',
 'Bed & Bath / Bedding / Sheets & Pillowcases / Twin XL Sheets & Pillowcases',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Serving Trays & Boards / Serving Trays & Platters / Serving Serving Trays & Platters',
 'Holiday Décor / Holiday Lighting',
 'Décor & Pillows / Wall Décor / Memo Boards',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Toilets & Bidets / Toilet Paper Holders / Wall Mounted Toilet Paper Holders',
 'Décor & Pillows / Window Treatments / Curtains & Drapes / 63 Inch and Less Curtains & Drapes',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Knobs / Egg Door Knobs',
 'Décor & Pillows / Clocks / Wall Clocks / Analog Wall Clocks',
 'Home Improvement / Doors & Door Hardware / Interior Doors / Sliding Interior Doors',
 'Outdoor / Outdoor Recreation / Outdoor Games / All Outdoor Games',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Levers / Round Door Levers',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Sheds / Storage Sheds',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Levers',
 'School Furniture and Supplies / School Furniture / School Tables / Folding Tables / Wood Folding Tables',
 'Décor & Pillows / Wall Décor / Wall Accents / Green Wall Accents',
 'School Furniture and Supplies / Facilities & Maintenance / Commercial Signage',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Garage Storage Cabinets',
 'Furniture / Bedroom Furniture / Dressers & Chests / Beige Dressers & Chests',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves',
 'Furniture / Game Tables & Game Room Furniture / Dartboards & Cabinets',
 'Outdoor / Outdoor Décor / Outdoor Pillows & Cushions / Patio Furniture Cushions / Lounge Chair Patio Furniture Cushions',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Dining Sets / Two Person Patio Dining Sets',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Ivory & Cream Throw Pillows',
 'Appliances / Washers & Dryers / Washer & Dryer Sets / Black Washer & Dryer Sets',
 'School Furniture and Supplies / School Furniture / School Chairs & Seating / Stackable Chairs',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Brass Cabinet & Drawer Pulls',
 'School Furniture and Supplies / School Boards & Technology / AV, Mounts & Tech Accessories / Electronic Mounts & Stands / Computer Mounts',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Papasan Accent Chairs',
 'Storage & Organization / Shoe Storage / All Shoe Storage / Rack Shoe Storage',
 'Storage & Organization / Shoe Storage / All Shoe Storage / Cabinet Shoe Storage',
 'Storage & Organization / Storage Containers & Drawers / Storage Drawers',
 'Appliances / Kitchen Appliances / Wine & Beverage Coolers / Water Coolers',
 'Furniture / Living Room Furniture / Chairs & Seating / Rocking Chairs',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Serving Bowls & Baskets / Serving Bowls / NA Serving Bowls',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / Projection Screens / Inflatable Projection Screens',
 'Appliances / Kitchen Appliances / Large Appliance Parts & Accessories',
 'Storage & Organization / Bathroom Storage & Organization / Hampers & Laundry Baskets / Laundry Hampers & Laundry Baskets',
 'Furniture / Office Furniture / Office Stools',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Outdoor Club Chairs / Metal Outdoor Club Chairs',
 'School Furniture and Supplies / School Furniture / School Tables / Folding Tables',
 'Lighting / Wall Lights / Bathroom Vanity Lighting / Traditional Bathroom Vanity Lighting',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Centerset Bathroom Sink Faucets',
 'Décor & Pillows / Flowers & Plants / Faux Flowers / Orchid Faux Flowers',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Metal Floor Tiles & Wall Tiles',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Sinks',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Outdoor Covers / Grill Covers / Charcoal Grill Grill Covers',
 'Outdoor / Outdoor Décor / Outdoor Wall Décor',
 'Storage & Organization / Cleaning & Laundry Organization / Laundry Room Organizers',
 'Reception Area / Reception Seating / Reception Sofas & Loveseats',
 'Kitchen & Tabletop / Cookware & Bakeware / Baking Sheets & Pans / Bread & Loaf Pans / Steel Bread & Loaf Pans',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Wingback Accent Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads / Fixed Shower Heads',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Kitchen Gadgets / Pasta Makers & Accessories',
 'School Furniture and Supplies / School Furniture / School Chairs & Seating / Classroom Chairs / High School & College Classroom Chairs',
 'Furniture / Living Room Furniture / Sectionals / Stationary Sectionals',
 'Furniture / Kitchen & Dining Furniture / Sideboards & Buffets / Drawer Equipped Sideboards & Buffets',
 'Kitchen & Tabletop / Cookware & Bakeware / Baking Sheets & Pans / Bread & Loaf Pans',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Cooking Utensils / All Cooking Utensils / Kitchen Cooking Utensils',
 'Décor & Pillows / Flowers & Plants / Live Plants',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / Projection Screens / Folding Frame Projection Screens',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Metal Kitchen Canisters & Jars',
 'Outdoor / Outdoor Décor / Outdoor Fountains',
 'Outdoor / Outdoor Shades / Pergolas / Wood Pergolas',
 'Décor & Pillows / Candles & Holders / Candle Holders / Sconce Candle Holders',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Cake & Tiered Stands',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Chrome Kitchen Faucets',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / White Throw Pillows',
 'Outdoor / Outdoor Fencing & Flooring / Turf',
 'Décor & Pillows / Window Treatments / Valances & Kitchen Curtains',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Knobs / Black Cabinet & Drawer Knobs',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Bronze Kitchen Faucets',
 'Appliances / Washers & Dryers / Washer & Dryer Sets',
 'Décor & Pillows / Clocks / Mantel & Tabletop Clocks',
 'Home Improvement / Doors & Door Hardware / Interior Doors',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves / Floating Wall & Display Shelves',
 'Outdoor / Outdoor Recreation / Backyard Play / Climbing Toys & Slides',
 'Home Improvement / Building Equipment / Dollies / Hand Truck Dollies',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Baby & Kids Dressers',
 'Décor & Pillows / Mirrors / All Mirrors / Leaning & Floor Mirrors',
 'Kitchen & Tabletop / Tableware & Drinkware / Drinkware / Mugs & Teacups',
 'Décor & Pillows / Flowers & Plants / Wreaths',
 'Outdoor / Outdoor Shades / Pergolas / Metal Pergolas',
 'Bed & Bath / Bedding / Sheets & Pillowcases / Twin Sheets & Pillowcases',
 'Outdoor / Outdoor Shades / Pergolas',
 'Reception Area / Reception Seating / Office Sofas & Loveseats',
 'Décor & Pillows / Home Accessories / Indoor Fountains',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Ceramic Kitchen Canisters & Jars',
 'Décor & Pillows / Window Treatments / Curtain Hardware & Accessories / Bracket Curtain Hardware & Accessories',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Accent Tiles / Ceramic Accent Tiles',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Accent Tiles',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Arm Accent Chairs',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Free Form Coffee Tables',
 'Décor & Pillows / Flowers & Plants / Faux Flowers / Rose Faux Flowers',
 'Bed & Bath / Mattresses & Foundations / Innerspring Mattresses / Twin Innerspring Mattresses',
 'Outdoor / Outdoor Décor / Outdoor Pillows & Cushions / Patio Furniture Cushions / Dining Chair Patio Furniture Cushions',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / TV Stands & Entertainment Centers / Traditional TV Stands & Entertainment Centers',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Plant Stands & Tables / Square Plant Stands & Tables',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves / Corner Wall & Display Shelves',
 "Rugs / Area Rugs / 3' x 5' Area Rugs",
 'Kitchen & Tabletop / Tableware & Drinkware / Drinkware / Mugs & Teacups / Coffee Mugs & Teacups',
 'Contractor / Entry & Hallway / Coat Racks & Umbrella Stands / Wall Mounted Coat Racks & Umbrella Stands',
 "Baby & Kids / Toddler & Kids Playroom / Indoor Play / Kids' Playhouses",
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Square Coffee Tables',
 'Baby & Kids / Toddler & Kids Playroom / Indoor Play / Dollhouses & Accessories',
 'Bed & Bath / Bedding / All Bedding / Queen Bedding',
 'No Classification Fits'
]

classifications_list = sorted(get_args(FullyQualifiedClassifications))

known_categories = set([c.split(" / ")[0].strip() for c in classifications_list])
known_sub_categories = set([c.split(" / ")[1].strip() for c in classifications_list if len(c.split(" / ")) > 1])

known_sub_categories


class Query(BaseModel):
    """
    Base model for search queries, containing common query attributes.
    """
    keywords: str = Field(
        ...,
        description="The original search query keywords sent in as input"
    )


class QueryClassification(Query):
    """
    Structured representation of a search query for furniture e-commerce.
    Inherits keywords from the base Query model and adds category and sub-category.
    """
    classifications: list[FullyQualifiedClassifications] = Field(
        description="A possible classification for the product."
    )

    @property
    def categories(self):
        return set([c.split(" / ")[0] for c in self.classifications])

    @property
    def sub_categories(self):
        return set([c.split(" / ")[1] for c in self.classifications if len(c.split(" / ")) > 1])



### Query classification code

In [ ]:
enricher = AutoEnricher(
     model="openai/gpt-4o",
     system_prompt="You are a helpful furniture shopping agent that helps users construct search queries.",
     response_model=QueryClassification
)

def get_prompt_fully_qualified(query):
        prompt = f"""
        As a helpful agent, you'll recieve requests from users looking for furniture products.

        Your task is to search with a structured query against a furniture product catalog.

        Here is the users request:

        {query}

        Return the best classifications for this user's query.

        Try to pick as diverse a set of possible to ensure the customer finds what they need.
        (IE different top level categories are better than very similar classifications that share most of their tree / subtree)

        Return an empty list if no classification fits, or its too ambiguous.

        """

        return prompt

def fully_classified(query):
    prompt = get_prompt_fully_qualified(query)
    classification = enricher.enrich(prompt).model_copy()
    if "No Classification Fits" in classification.classifications:
        classification.classifications = []
    return classification

fully_classified("dinooosaur"), fully_classified("sofa loveseat")

2025-08-31 21:07:50,917 - data_dir - INFO - Looking for openai in environment variables or globals...


INFO:data_dir:Looking for openai in environment variables or globals...


2025-08-31 21:07:50,918 - data_dir - INFO - Reading openai_api_key API key from /content/drive/MyDrive/cheat-at-search-data//keys.json


INFO:data_dir:Reading openai_api_key API key from /content/drive/MyDrive/cheat-at-search-data//keys.json


2025-08-31 21:07:51,147 - data_dir - INFO - openai_api_key key loaded successfully.


...

2025-08-31 21:07:53,509 - query_parser - ERROR - Error loading cache file /content/drive/MyDrive/cheat-at-search-data/enrich_cache/openaienricher_3827e3cfb0a48ec60363a6165c09cbd1_cache.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


ERROR:query_parser:Error loading cache file /content/drive/MyDrive/cheat-at-search-data/enrich_cache/openaienricher_3827e3cfb0a48ec60363a6165c09cbd1_cache.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


2025-08-31 21:07:53,511 - query_parser - ERROR - Starting with empty cache due to error.


ERROR:query_parser:Starting with empty cache due to error.


(QueryClassification(keywords='dinooosaur', classifications=[]),
 QueryClassification(keywords='sofa loveseat', classifications=['Furniture / Living Room Furniture / Sofas', 'Reception Area / Reception Seating / Reception Sofas & Loveseats', 'Reception Area / Reception Seating / Office Sofas & Loveseats']))

### Redefine ground truth

In [ ]:
from cheat_at_search.wands_data import labeled_query_products, queries

def get_top_categories(column, no_fit_label, cutoff=0.8):
    # Get relevant products per query
    top_products = labeled_query_products[labeled_query_products['grade'] == 2]

    # Aggregate top categories
    categories_per_query_ideal = top_products.groupby('query')[column].value_counts().reset_index()

    # Get as percentage of all categories for this query
    top_cat_proportion = categories_per_query_ideal.groupby(['query', column]).sum() / categories_per_query_ideal.groupby('query').sum()
    top_cat_proportion = top_cat_proportion.drop(columns=column).reset_index()

    # Only look at cases where the category is > 0.8
    top_cat_proportion = top_cat_proportion[top_cat_proportion['count'] > cutoff]
    top_cat_proportion[column].fillna(no_fit_label, inplace=True)
    ground_truth_cat = top_cat_proportion
    # Give No Category Fits to all others without dominant category
    ground_truth_cat = ground_truth_cat.merge(queries, how='right', on='query')[['query', column, 'count']]
    ground_truth_cat[column].fillna(no_fit_label, inplace=True)
    return ground_truth_cat

# The original ground truth by category
ground_truth_cat = get_top_categories('category', 'No Category Fits', cutoff=0.8)


ground_truth_cat_list = get_top_categories('category', 'No Category Fits', cutoff=0.05)

# Group by query, collect category into list
ground_truth_cat_list = ground_truth_cat_list.groupby('query').agg({'category': list}).reset_index()
# Remove empty items on "No Category Fits" from lists
ground_truth_cat_list['category'] = ground_truth_cat_list['category'].apply(lambda x: [y.strip() for y in x if y and y != 'No Category Fits'])
ground_truth_cat_list

/tmp/ipython-input-2868813019.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  top_cat_proportion[column].fillna(no_fit_label, inplace=True)
/tmp/ipython-input-2868813019.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

,query,category
0,12v dirt bike,[]
1,3 1/2 inch drawer pull,[Home Improvement]
2,3 piece rug set with runners,[Rugs]
3,3/4 size mattress,[]
4,30 inch bathroom vanity,[Home Improvement]
...,...,...
475,wrought tv stand,[Furniture]
476,zachary 72.5,[Furniture]
477,zakariyah twin platform,[Baby & Kids]
478,zen,"[Décor & Pillows, Home Improvement, Outdoor]"


In [ ]:
def get_preds(cat, column):
    if column == 'category':
        return cat.categories
    elif column == 'sub_category':
        return cat.sub_categories
    else:
        raise ValueError(f"Unknown column {column}")


def jaccard_sim(ground_truth, column, no_fit_label, classifier_fn):

    jaccard_sum = 0
    num_preds = 0
    for _, row in ground_truth.iterrows():
        query = row['query']
        expected_category = set(row[column])
        cat = classifier_fn(query)
        preds = set(get_preds(cat, column))
        if (len(preds) == 0) or (preds == [no_fit_label]):
            print(f"Skipping {query}")
            continue

        jaccard = len(preds.intersection(expected_category)) / len(preds.union(expected_category))
        print(f"{query} -- pred:{preds} == expected:{expected_category}")
        print(f"Jaccard: {jaccard}")
        jaccard_sum += jaccard
        num_preds += 1
    return jaccard_sum / num_preds, num_preds / len(ground_truth)


jaccard, coverage = jaccard_sim(ground_truth_cat_list, 'category', 'No Category Fits', fully_classified)
jaccard, coverage

Skipping 12v dirt bike
3 1/2 inch drawer pull -- pred:{'Home Improvement'} == expected:{'Home Improvement'}
Jaccard: 1.0
3 piece rug set with runners -- pred:{'Rugs'} == expected:{'Rugs'}
Jaccard: 1.0
3/4 size mattress -- pred:{'Bed & Bath'} == expected:set()
Jaccard: 0.0
30 inch bathroom vanity -- pred:{'Home Improvement'} == expected:{'Home Improvement'}
Jaccard: 1.0
46 inch closet door -- pred:{'Home Improvement'} == expected:{'Home Improvement'}
Jaccard: 1.0
48 in entry table with side by side drawer -- pred:{'Furniture'} == expected:{'Furniture'}
Jaccard: 1.0
48 inch bathroom vanity with trough sink -- pred:{'Home Improvement'} == expected:set()
Jaccard: 0.0
48 inch kitchen hood -- pred:{'Appliances'} == expected:{'Appliances'}
Jaccard: 1.0
48" sliding single track , barn door for laundry -- pred:{'Home Improvement'} == expected:{'Home Improvement', 'Sale'}
Jaccard: 0.5
5 gang light switch -- pred:{'Home Improvement'} == expected:set()
Jaccard: 0.0
54 in bench cushion -- pred:{'Ou

(0.6449576597382601, 0.9020833333333333)

### Maybe a better question -- do we recall the most important prediction?

In [ ]:
def recall_when_pred(categorized, ground_truth_cat):
    """When we retrieve a category, is it correct?"""
    hits = []
    misses = []
    idx = 0
    for _, row in ground_truth_cat.iterrows():
        query = row['query']
        expected_category = row['category']

        cat = categorized(query)
        if len(cat.classifications) == 0:
            print(f"{idx} Skipping {query}")
            continue
        if expected_category != "No Category Fits":
            if expected_category.strip() in cat.categories:
                print(f"{idx} q:{query} -- pred:{cat.categories} == expected:{expected_category.strip()}")
                hits.append((expected_category, cat))
            else:
                print("***")
                print(f"{query} -- pred:{cat.categories} != expected:{expected_category.strip()}")
                print(cat.classifications)
                misses.append((expected_category, cat))
                num_so_far = len(hits) + len(misses)
                print(f"{idx} recall (N={num_so_far}) -- {len(hits) / (len(hits) + len(misses))}")
        idx += 1

    return len(hits) / (len(hits) + len(misses)), hits, misses

prec, hits, misses = recall_when_pred(fully_classified, ground_truth_cat)
prec

1 q:smart coffee table -- pred:{'Furniture'} == expected:Furniture
2 Skipping dinosaur
2 q:turquoise pillows -- pred:{'Décor & Pillows'} == expected:Décor & Pillows
3 q:chair and a half recliner -- pred:{'Furniture'} == expected:Furniture
4 q:sofa with ottoman -- pred:{'Furniture'} == expected:Furniture
6 q:driftwood mirror -- pred:{'Décor & Pillows'} == expected:Décor & Pillows
7 q:home sweet home sign -- pred:{'Décor & Pillows'} == expected:Décor & Pillows
***
coffee table fire pit -- pred:{'Outdoor'} != expected:
['Outdoor / Outdoor & Patio Furniture / Outdoor Tables / All Patio Tables', 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Conversation Sets', 'Outdoor / Outdoor Décor / Outdoor Fountains']
8 recall (N=7) -- 0.8571428571428571
9 q:king poster bed -- pred:{'Furniture'} == expected:Furniture
10 q:ombre rug -- pred:{'Rugs'} == expected:Rugs
11 q:large spoon and fork wall decor -- pred:{'Décor & Pillows'} == expected:Décor & Pillows
12 q:outdoor privacy wal

0.9423728813559322

In [ ]:
def recall_all(categorized, ground_truth):
    """When we retrieve a category, is it correct?"""
    hits = []
    misses = []
    idx = 0
    for _, row in ground_truth.sample(frac=1).iterrows():
        query = row['query']
        expected_category = row['category']

        cat = categorized(query)
        if len(cat.classifications) == 0:
            print(f"{idx} Skipping {query}")
            continue
        # ***
        # Now also consider this a miss, we should not have predicted any caterogies
        if expected_category == "No Category Fits" and len(cat.categories) > 0:
            print("!**")
            print(f"{query} -- pred:{cat.categories} != expected:{expected_category.strip()}")
            print(cat.classifications)
            misses.append((expected_category, cat))
            num_so_far = len(hits) + len(misses)
            print(f"{idx} recall (N={num_so_far}) -- {len(hits) / (len(hits) + len(misses))}")
        if expected_category.strip() in cat.categories:
            print(f"{idx} q:{query} -- pred:{cat.categories} == expected:{expected_category.strip()}")
            hits.append((expected_category, cat))
        else:
            print("***")
            print(f"{query} -- pred:{cat.categories} != expected:{expected_category.strip()}")
            print(cat.classifications)
            misses.append((expected_category, cat))
            num_so_far = len(hits) + len(misses)
            print(f"{idx} recall (N={num_so_far}) -- {len(hits) / (len(hits) + len(misses))}")
        idx += 1

    return len(hits) / (len(hits) + len(misses)), hits, misses

recall, hits, misses = recall_all(fully_classified, ground_truth_cat)
recall

0 q:hussain handmade tufted magenta rug -- pred:{'Rugs'} == expected:Rugs
1 q:golub dining table -- pred:{'Furniture'} == expected:Furniture
2 q:hulmeville writing desk with hutch -- pred:{'Furniture'} == expected:Furniture
3 q:tile backsplash -- pred:{'Home Improvement'} == expected:Home Improvement
4 q:ombre rug -- pred:{'Rugs'} == expected:Rugs
!**
acrylic clear chair -- pred:{'Furniture'} != expected:No Category Fits
['Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs', 'Furniture / Office Furniture / Office Chairs', 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs']
5 recall (N=6) -- 0.8333333333333334
***
acrylic clear chair -- pred:{'Furniture'} != expected:No Category Fits
['Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs', 'Furniture / Office Furniture / Office Chairs', 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs']
5 recall (N=7) -- 0.7142857142857

0.4868651488616462

## Run Category search strategy with classifier

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np


class CategorySearch(SearchStrategy):
    def __init__(self, products, query_to_cat,
                 name_boost=9.3,
                 description_boost=4.1,
                 category_boost=10,
                 sub_category_boost=5):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        products['category'] = cat_split.apply(
            lambda x: x[0].strip() if len(x) > 0 else ""
        )
        products['subcategory'] = cat_split.apply(
            lambda x: x[1].strip() if len(x) > 1 else ""
        )
        self.index['category_snowball'] = SearchArray.index(
            products['category'], snowball_tokenizer
        )
        self.index['subcategory_snowball'] = SearchArray.index(
            products['subcategory'], snowball_tokenizer
        )

        self.query_to_cat = query_to_cat
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.category_boost = category_boost
        self.sub_category_boost = sub_category_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        structured = self.query_to_cat(query)
        tokenized = snowball_tokenizer(query)

        # ****
        # Baseline BM25 search from before
        for token in tokenized:
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ****
        # If there's a subcategory, boost that by a constant amount
        for sub_category in structured.sub_categories:
            tokenized_subcategory = snowball_tokenizer(sub_category)
            subcategory_match = np.zeros(len(self.index))
            if tokenized_subcategory:
                subcategory_match = self.index['subcategory_snowball'].array.score(tokenized_subcategory) > 0
            bm25_scores[subcategory_match] += self.sub_category_boost

        # ****
        # If there's a category, boost that by a constant amount
        for category in structured.categories:
            tokenized_category = snowball_tokenizer(category)
            category_match = np.zeros(len(self.index))
            if tokenized_category:
                category_match = self.index['category_snowball'].array.score(tokenized_category) > 0
            bm25_scores[category_match] += self.category_boost

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores


In [ ]:
categorized_search = CategorySearch(products, fully_classified)
graded_categorized = run_strategy(categorized_search)
graded_categorized

2025-08-31 21:26:06,889 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-08-31 21:26:06,909 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-08-31 21:26:06,912 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

2025-08-31 21:26:20,418 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-08-31 21:26:20,448 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete
Searching: 100%|██████████| 480/480 [00:12<00:00, 37.28it/s]


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,query_id,rank,query_class,id,label,grade,discounted_gain,idcg,dcg,ndcg
0,7465,hair salon chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,offers a wide selection of professional salon ...,fauxleathertype : pu|legheight-toptobottom:18|...,69.0,4.5,53.0,"[fauxleathertype : pu, legheight-toptobottom:1...",...,0,1,Massage Chairs,80.0,Exact,2.0,3.00,8.786905,8.586905,0.977239
1,25431,barberpub salon massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,supplierintendedandapproveduse : non residenti...,4.0,5.0,4.0,[supplierintendedandapproveduse : non resident...,...,0,2,Massage Chairs,29.0,Exact,2.0,1.50,8.786905,8.586905,0.977239
2,7468,mercer41 hair salon chair hydraulic styling ch...,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,mercer41 beauty offers a wide selection profes...,seatfillmaterial : foam|waterrepellant : no re...,1.0,5.0,1.0,"[seatfillmaterial : foam, waterrepellant : no ...",...,0,3,Massage Chairs,104.0,Exact,2.0,1.00,8.786905,8.586905,0.977239
3,39461,professional salon reclining massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,new and in a good condition . first-rate metal...,overalldepth-fronttoback:39.4|warrantylength:1...,NaN,NaN,NaN,"[overalldepth-fronttoback:39.4, warrantylength...",...,0,4,Massage Chairs,114.0,Exact,2.0,0.75,8.786905,8.586905,0.977239
4,39428,barber salon reclining massage chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,heavy-duty hydraulic recline barber chair salo...,seatwidth-sidetoside:19|upholsterycolor : blac...,1.0,5.0,0.0,"[seatwidth-sidetoside:19, upholsterycolor : bl...",...,0,5,Massage Chairs,25.0,Exact,2.0,0.60,8.786905,8.586905,0.977239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,22194,wine glass rack,Kitchen Sink Storage,Kitchen & Tabletop / Kitchen Organization / Co...,drip-dry up to eight wineglasses with this cle...,glasscapacity:8|countryoforigin : united state...,5.0,4.5,3.0,"[glasscapacity:8, countryoforigin : united sta...",...,487,6,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4796,40243,madisen hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,complement your farmhouse kitchen decor with t...,producttype : wine glass rack|overallwidth-sid...,29.0,5.0,20.0,"[producttype : wine glass rack, overallwidth-s...",...,487,7,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4797,40244,kena hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,spruce up your farmhouse kitchen decor with th...,warrantylength:1 year|producttype : wine glass...,23.0,5.0,18.0,"[warrantylength:1 year, producttype : wine gla...",...,487,8,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4798,39976,wall mounted wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,"the latest addition to this collection , this ...",overallheight-toptobottom:4|design : wall moun...,34.0,4.5,18.0,"[overallheight-toptobottom:4, design : wall mo...",...,487,9,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000


In [ ]:
ndcgs(graded_bm25).mean(), ndcgs(graded_categorized).mean()

(np.float64(0.5411098691836396), np.float64(0.5662156211895406))

In [ ]:
deltas = ndcg_delta(graded_categorized, graded_bm25)

In [ ]:
sig_improved = len(deltas[deltas > 0.1])
print(f"Num Significatly Improved: {sig_improved}")
deltas[deltas > 0.1]

Num Significatly Improved: 47


,ndcg
query,
drum picture,0.506435
bathroom vanity knobs,0.486519
non slip shower floor tile,0.477081
wood rack wide,0.469449
outdoor lounge chair,0.456442
modern outdoor furniture,0.365533
twin bed frame,0.359391
podium with locking cabinet,0.332656
outdoor light fixtures,0.320553


In [ ]:
sig_harmed = len(deltas[deltas < -0.1])
print(f"Num Significatly Harmed: {sig_harmed}")
print(f"Prop improved/harmed: {sig_improved / (sig_harmed + sig_improved)} | {sig_harmed / (sig_harmed + sig_improved)}")
deltas[deltas < -0.1]

Num Significatly Harmed: 4
Prop improved/harmed: 0.9215686274509803 | 0.0784313725490196


,ndcg
query,
zodiac pillow,-0.113806
papasan chair frame only,-0.123290
outdoor lounge cushions,-0.223547
sugar canister,-0.224721


### Look at a query

In [ ]:
QUERY = "sugar canister"
fully_classified(QUERY)

QueryClassification(keywords='sugar canister', classifications=['Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Ceramic Kitchen Canisters & Jars', 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Metal Kitchen Canisters & Jars'])

In [ ]:
ground_truth_cat[ground_truth_cat['query'] == QUERY]

,query,category,count
183,sugar canister,,0.847458


In [ ]:
graded_categorized[graded_categorized['query'] == QUERY][['product_name', 'category hierarchy', 'grade']]

,product_name,category hierarchy,grade
1830,"everyday coffee , sugar , flour kitchen canist...",Kitchen & Tabletop / Kitchen Organization / Fo...,2.0
1831,kitchen canister,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1832,kitchen canister,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1833,wave 4 piece kitchen canister set,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1834,enameled metal `` sugar '' kitchen canister,NaN,2.0
1835,4.69 qt . kitchen canister,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1836,tower salt and sugar container 2 jar spice rack,Kitchen & Tabletop / Kitchen Organization / Sp...,1.0
1837,"metal 3 piece sugar , flour & coffee canister set",NaN,2.0
1838,pauline 3 piece kitchen canister set,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1839,versailles 4 piece kitchen canister set,Kitchen & Tabletop / Kitchen Organization / Fo...,2.0


In [ ]:
graded_bm25[graded_bm25['query'] == QUERY][['product_name', 'category hierarchy', 'grade']]

,product_name,category hierarchy,grade
1830,"everyday coffee , sugar , flour kitchen canist...",Kitchen & Tabletop / Kitchen Organization / Fo...,2.0
1831,enameled metal `` sugar '' kitchen canister,NaN,2.0
1832,"metal 3 piece sugar , flour & coffee canister set",NaN,2.0
1833,kitchen canister,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1834,4 qt . sugar jar,NaN,2.0
1835,kitchen canister,Kitchen & Tabletop / Kitchen Organization / Fo...,1.0
1836,coffee & food storage canister,NaN,2.0
1837,4 piece kitchen canister set,NaN,2.0
1838,tuscania : tuscania canister zucchero ( sugar ),NaN,2.0
1839,3 qt . kitchen canister,NaN,1.0


In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np

from cheat_at_search.agent.enrich import CachedEnricher, OpenAIEnricher


class CategorySearch(SearchStrategy):
    def __init__(self, products, query_to_cat,
                 name_boost=9.3,
                 description_boost=4.1,
                 category_boost=10,
                 sub_category_boost=5):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        products['category'] = cat_split.apply(
            lambda x: x[0].strip() if len(x) > 0 else ""
        )
        products['subcategory'] = cat_split.apply(
            lambda x: x[1].strip() if len(x) > 1 else ""
        )
        self.index['category_snowball'] = SearchArray.index(
            products['category'], snowball_tokenizer
        )
        self.index['subcategory_snowball'] = SearchArray.index(
            products['subcategory'], snowball_tokenizer
        )

        self.query_to_cat = query_to_cat
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.category_boost = category_boost
        self.sub_category_boost = sub_category_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        structured = self.query_to_cat(query)
        tokenized = snowball_tokenizer(query)

        # ****
        # Baseline BM25 search from before
        num_tokens_matched = np.zeros(len(self.index))
        for token in tokenized:

            name_score = self.index['product_name_snowball'].array.score(token) * self.name_boost
            desc_score = self.index['product_description_snowball'].array.score(
                token) * self.description_boost
            bm25_scores += name_score + desc_score
            num_tokens_matched[(name_score + desc_score) > 0] += 1

        # ****
        # If there's a subcategory, boost that by a constant amount
        for sub_category in structured.sub_categories:
            tokenized_subcategory = snowball_tokenizer(sub_category)
            subcategory_match = np.zeros(len(self.index))
            if tokenized_subcategory:
                subcategory_match = self.index['subcategory_snowball'].array.score(tokenized_subcategory) > 0
            bm25_scores[subcategory_match] += self.sub_category_boost

        # ****
        # If there's a category, boost that by a constant amount
        for category in structured.categories:
            tokenized_category = snowball_tokenizer(category)
            category_match = np.zeros(len(self.index))
            if tokenized_category:
                category_match = self.index['category_snowball'].array.score(tokenized_category) > 0
            bm25_scores[category_match] += self.category_boost

        # ***
        # Require all tokens to match
        min_token_match = max(len(tokenized) / 2, 1)
        bm25_scores[num_tokens_matched < min_token_match] *= 0.8

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores


In [ ]:
categorized_search = CategorySearch(products, fully_classified)
graded_categorized2 = run_strategy(categorized_search)
graded_categorized2

In [ ]:
ndcgs(graded_bm25).mean(), ndcgs(graded_categorized2).mean()

In [ ]:
categorized_search = CategorySearch(products, fully_classified, category_boost=100, sub_category_boost=50)
graded_categorized = run_strategy(categorized_search)
graded_categorized

In [ ]:
def get_num_tokens(query):
    prompt = get_prompt_fully_qualified(query)
    return enricher.get_num_tokens(prompt)

get_num_tokens("loveseat sofa")

In [ ]:
get_num_tokens("loveseat")